In [8]:
import pandas as pd
import string

In [5]:
data = pd.read_csv('fb-wx-data.csv')

In [6]:
data = data[['content', 'split', 'baidu_seg', 'baidu_prob', 'label']]

In [7]:
data['content_index'] = range(data.shape[0])

# Process 'split'

In [9]:
def split_text(text):
    length = len(text)
    text_block = []
    i, j = 0, 500

    while j < length:
        while text[j-1] != '。' and text[j-1] != '！':
            j -= 1
            if j == i:
                j = i + 500
                while text[j-1] != '，':
                    j -= 1
                    if j == i:
                        j = i + 500
                        break
                break
        text_block.append(text[i:j])
        i = j
        j = i + 500
    text_block.append(text[i:length])

    return text_block

In [10]:
for i in range(len(data.index)):
    data.at[i, 'split'] = split_text(data.at[i, 'content'])

# Process 'baidu_seg'

In [12]:
def seg_to_list(text):
    length = len(text)
    list_all = []
    temp_list = []

    i, j = 1, 1
    while j < length - 1:
        if text[i] == "'":
            j += 2
            while j < length - 1:
                if text[j] == "'" and (text[j+1] == ',' or text[j+1] == ']'):
                    temp_list.append(text[i+1:j])
                    i = j
                    break
                j += 1

        if text[j] == ']' and (text[j+1] == ',' or text[j+1] == ']'):
            list_all.append(temp_list)
            temp_list = []
            if text[j+1] == ']':
                break
            else: 
                i = j + 3
                j = i

        i += 1
        j += 1
    
    return list_all

In [13]:
for i in range(len(data.index)):
    data.at[i, 'baidu_seg'] = seg_to_list(data.at[i, 'baidu_seg'])

In [14]:
# Remove stopwords
stopwords = [line.strip() for line in open('stopwords.txt', encoding = 'UTF-8').readlines()]

In [16]:
def remove_stopwords(wordlist):
    output = [word for word in wordlist if word not in stopwords]
    return output

In [19]:
for i in range(len(data.index)):
    data.at[i, 'baidu_seg'] = [remove_stopwords(text) for text in data.at[i, 'baidu_seg']]

# Process 'baidu_prob'

In [21]:
def prob_to_list(text):
    length = len(text)
    list_all = []

    i, j = 0, 0
    while j < length - 1:
        if text[i].isdigit():
            while j < length:
                if text[j] == ',' or text[j] == ']':
                    list_all.append(float(text[i:j]))
                    i = j
                    break
                j += 1
        
        i += 1
        j += 1
    
    return list_all

In [22]:
for i in range(len(data.index)):
    data.at[i, 'baidu_prob'] = prob_to_list(data.at[i, 'baidu_prob'])

# View Data

In [23]:
data.head()

,content,split,baidu_seg,baidu_prob,label,content_index
0,本网讯（通讯员孙静娴）12月13日上午，中建一局房地产公司组织召开第七次中心组学习（扩大）会...,[本网讯（通讯员孙静娴）12月13日上午，中建一局房地产公司组织召开第七次中心组学习（扩大）...,"[[网讯, 通讯员, 孙静娴, 12月13日上午, 中建一局房地产公司, 组织, 召开, 第...","[0.00307655, 0.00122125, 0.00661374]",0.0,0
1,原标题：三项制度改革让“能者上、平者让、庸者下”11月11日，中建一局二公司生产组织中心主任...,[原标题：三项制度改革让“能者上、平者让、庸者下”11月11日，中建一局二公司生产组织中心主...,"[[原, 标题, 三项, 制度, 改革, 能者, 平者, 庸者, 11月11日, 中建一局二...","[0.240754, 0.918229, 0.0709911]",0.0,1
2,原标题：六家工地劳动用工标准受到通报表彰记者29日从郑州城乡建设局了解到，今年第四季度，有6...,[原标题：六家工地劳动用工标准受到通报表彰记者29日从郑州城乡建设局了解到，今年第四季度，有...,"[[原, 标题, 六家, 工地, 劳动, 用工, 标准, 通报, 表彰, 记者, 29日, ...",[0.0979948],0.0,2
3,合肥八院始建于1952年，俗称三康医院，已走过67年的辉煌历史，是一所集医疗、护理、保健、教...,[合肥八院始建于1952年，俗称三康医院，已走过67年的辉煌历史，是一所集医疗、护理、保健、...,"[[合肥八院, 始建于, 1952年, 俗称, 三康医院, 走过, 67年, 辉煌, 历史,...","[0.100506, 0.1208]",0.0,3
4,打造创新力，加强竞争力，提升发展力——近年来，中建一局二公司在全面加强党的领导的同时，立足企...,[打造创新力，加强竞争力，提升发展力——近年来，中建一局二公司在全面加强党的领导的同时，立足...,"[[打造, 创新, 力, 加强, 竞争力, 提升, 发展, 力, 近年, 中建一局二公司, ...","[0.0123384, 0.514408, 0.28322]",0.0,4


In [24]:
data.loc[15982]

content          各地也纷纷出台配套意见。过程结算要求：过程结算文件直接作为竣工结算依据；先结算，后竣工；不结...
split            [各地也纷纷出台配套意见。过程结算要求：过程结算文件直接作为竣工结算依据；先结算，后竣工；不...
baidu_seg        [[各地, 纷纷, 出台, 配套, 意见, 过程, 结算, 要求, 过程, 结算, 文件, ...
baidu_prob       [0.991483, 0.00862393, 0.047756, 0.00427403, 0...
label                                                          NaN
content_index                                                15982
Name: 15982, dtype: object

# Simplify

In [25]:
seg_list = []
for i in range(len(data.index)):
    for text in data.at[i, 'baidu_seg']:
        seg_list.append(text)

In [31]:
prob_list = []
content_list = []
content_index_list = []
for i in range(len(data.index)):
    content_i = data.loc[i,'content']
    content_index_i = data.loc[i,'content_index']
    for prob in data.at[i, 'baidu_prob']:
        prob_list.append(prob)
        content_list.append(content_i)
        content_index_list.append(content_index_i)

In [32]:
data_sim = pd.DataFrame({'baidu_seg': seg_list, 'baidu_prob': prob_list, 'content' : content_list, 'content_index':content_index_list})

In [33]:
len(data_sim.index)

66760

In [36]:
data_sim.to_csv('data_sim_with_content_and_idx.csv')